In [1]:
import tensorflow as tf
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow import keras

train_data, test_data = tfds.load(name='imdb_reviews', split=(tfds.Split.TRAIN,tfds.Split.TEST,), as_supervised=True)


In [2]:
type(train_data), type(test_data)

train_examples, train_labels = next(iter(train_data.batch(1)))

train_examples

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
      dtype=object)>

In [3]:
train_labels

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

In [4]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))


In [5]:
train_examples[:1]

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
      dtype=object)>

In [6]:
hub_layer(train_examples[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [9]:
model.fit(train_data.shuffle(1000).batch(512), epochs=20, verbose=1)

Epoch 1/20
49/49 [==============================] - 2s 29ms/step - loss: 1.0597 - accuracy: 0.5184
Epoch 2/20
49/49 [==============================] - 1s 27ms/step - loss: 0.6466 - accuracy: 0.6269
Epoch 3/20
49/49 [==============================] - 1s 27ms/step - loss: 0.5804 - accuracy: 0.7002
Epoch 4/20
49/49 [==============================] - 1s 28ms/step - loss: 0.5224 - accuracy: 0.7506
Epoch 5/20
49/49 [==============================] - 1s 27ms/step - loss: 0.4631 - accuracy: 0.7934
Epoch 6/20
49/49 [==============================] - 1s 27ms/step - loss: 0.4103 - accuracy: 0.8270
Epoch 7/20
49/49 [==============================] - 1s 27ms/step - loss: 0.3647 - accuracy: 0.8511
Epoch 8/20
49/49 [==============================] - 1s 27ms/step - loss: 0.3269 - accuracy: 0.8684
Epoch 9/20
49/49 [==============================] - 1s 27ms/step - loss: 0.2949 - accuracy: 0.8842
Epoch 10/20
49/49 [==============================] - 1s 27ms/step - loss: 0.2661 - accuracy: 0.8964
Epoch 11/

In [14]:
model.fit(test_data.shuffle(1000).batch(512), epochs=20, verbose=1)

Epoch 1/20
49/49 [==============================] - 1s 27ms/step - loss: 0.3084 - accuracy: 0.8744
Epoch 2/20
49/49 [==============================] - 1s 27ms/step - loss: 0.2609 - accuracy: 0.8936
Epoch 3/20
49/49 [==============================] - 1s 26ms/step - loss: 0.2343 - accuracy: 0.9064
Epoch 4/20
49/49 [==============================] - 1s 26ms/step - loss: 0.2154 - accuracy: 0.9148
Epoch 5/20
49/49 [==============================] - 1s 26ms/step - loss: 0.2001 - accuracy: 0.9226
Epoch 6/20
49/49 [==============================] - 1s 26ms/step - loss: 0.1874 - accuracy: 0.9300
Epoch 7/20
49/49 [==============================] - 1s 26ms/step - loss: 0.1755 - accuracy: 0.9354
Epoch 8/20
49/49 [==============================] - 1s 26ms/step - loss: 0.1652 - accuracy: 0.9407
Epoch 9/20
49/49 [==============================] - 1s 26ms/step - loss: 0.1558 - accuracy: 0.9448
Epoch 10/20
49/49 [==============================] - 1s 26ms/step - loss: 0.1475 - accuracy: 0.9494
Epoch 11/

In [15]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.0748 - accuracy: 0.9809
loss: 0.075
accuracy: 0.981
